<a href="https://colab.research.google.com/github/iyves/ru_col_suggest/blob/master/attest_collocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This colab shows how to lemmatize and get replacements for collocations.

In [1]:
# Initialize the colab - clone the github repository & install the dependencies
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Note: This file path may require modification to work with the active account's gdrive file structure
%cd drive/MyDrive/
! git clone https://github.com/iyves/ru_col_suggest.git

/content/drive/MyDrive
fatal: destination path 'ru_col_suggest' already exists and is not an empty directory.


In [9]:
%cd /content/drive/MyDrive/ru_col_suggest/src/models

# Downloading the tokenization models
!pip install ufal.udpipe
!pip install treetaggerwrapper

# Note: Make sure to update the version to the latest (see https://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/)
!wget https://www.cis.lmu.de/~schmid/tools/TreeTagger/data/tree-tagger-linux-3.2.4.tar.gz
!wget https://www.cis.lmu.de/~schmid/tools/TreeTagger/data/tagger-scripts.tar.gz
!wget https://www.cis.lmu.de/~schmid/tools/TreeTagger/data/install-tagger.sh
!wget http://corpus.leeds.ac.uk/mocky/russian.par.gz
!sh install-tagger.sh
!wget http://corpus.leeds.ac.uk/mocky/ru-table.tab

# Installation for windows: https://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/#Windows
# Move the bin, lib, and cmd folders to the src/models folder

/content/drive/MyDrive/ru_col_suggest/src/models
--2021-07-25 06:35:57--  https://www.cis.lmu.de/~schmid/tools/TreeTagger/data/tree-tagger-linux-3.2.4.tar.gz
Resolving www.cis.lmu.de (www.cis.lmu.de)... 129.187.148.72, 2001:4ca0:4f01::5
Connecting to www.cis.lmu.de (www.cis.lmu.de)|129.187.148.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1889240 (1.8M) [application/x-gzip]
Saving to: ‘tree-tagger-linux-3.2.4.tar.gz.4’

tree-tagger-linux-3 100%[===================>]   1.80M  2.37MB/s    in 0.8s    

2021-07-25 06:35:58 (2.37 MB/s) - ‘tree-tagger-linux-3.2.4.tar.gz.4’ saved [1889240/1889240]

--2021-07-25 06:35:58--  https://www.cis.lmu.de/~schmid/tools/TreeTagger/data/tagger-scripts.tar.gz
Resolving www.cis.lmu.de (www.cis.lmu.de)... 129.187.148.72, 2001:4ca0:4f01::5
Connecting to www.cis.lmu.de (www.cis.lmu.de)|129.187.148.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 107396 (105K) [application/x-gzip]
Saving to: ‘tagger-sc

In [1]:
# If the github is already cloned, only this cell must be run after restarting the runtime
# Note: be sure to mount the gdrive before continuing
%cd /content/drive/MyDrive/ru_col_suggest/
!git fetch
!git pull
# !git reset --hard origin/master

!pip install wget

# For static word embedding models
!pip install glove-python-binary
!pip install gensim

# For dynamic word embedding models
!pip uninstall -y tensorflow
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'

# For attesting collocations
!pip install mysql-connector-python

# Note: be sure to update the config.ini file and restart the runtime

/content/drive/MyDrive/ru_col_suggest
Already up to date.
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-xft9d1rr
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-xft9d1rr
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
tokenizers                    0.10.3
transformers                  4.10.0.dev0


## Lemmatization of collocations

In [ ]:
from src.tokenizer import Tokenizer

input_collocations = ["исследуем вопрос", "по мнению авторов"]

# Lemmatization via UDPipe
print("\n"*2, "Lemmatization via UDPipe")
tokenizer = Tokenizer(Tokenizer.Method.UDPIPE)
udpipe_output = tokenizer.tokenize(input_collocations)
for inp, outp in zip(input_collocations, udpipe_output):
  print(inp, "->", outp)


# Lemmatization via TreeTagger
print("\n"*2, "Lemmatization via TreeTagger")
tokenizer = Tokenizer(Tokenizer.Method.TREETAGGER)
treetagger_output = tokenizer.tokenize(input_collocations)
for inp, outp in zip(input_collocations, treetagger_output):
  print(inp, "->", outp)

/usr/local/lib/python3.7/dist-packages/treetaggerwrapper.py:740: FutureWarning: Possible nested set at position 8
  re.IGNORECASE | re.VERBOSE)
/usr/local/lib/python3.7/dist-packages/treetaggerwrapper.py:2044: FutureWarning: Possible nested set at position 152
  re.VERBOSE | re.IGNORECASE)
/usr/local/lib/python3.7/dist-packages/treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
/usr/local/lib/python3.7/dist-packages/treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)

Loading the model from /content/drive/MyDrive/ru_col_suggest/src/models/udpipe_syntagrus.model...




 Lemmatization via UDPipe
исследуем вопрос -> исследовать_VERB вопрос_NOUN
по мнению авторов -> по_ADP мнение_NOUN автор_NOUN


 Lemmatization via TreeTagger
исследуем вопрос -> исследовать_V вопрос_N
по мнению авторов -> по_S мнение_N автор_N


## Getting suggested collocations from static word embeddings (w2v, fastText, GloVe)

In [4]:
from src.static_embedder import LossLogger, StaticEmbedder

w2v_path = "/content/drive/MyDrive/models/lemma/w2v/w2v_treetagger.model"
fasttext_path = "/content/drive/MyDrive/models/lemma/fastText/fastText_treetagger.model"
glove_path = "/content/drive/MyDrive/models/lemma/glove/glove_treetagger.txt.word2vec"

# Load static word embedding models pretrained on a treetagger-lemmatized corpus
w2v_model = StaticEmbedder(StaticEmbedder.Model.WORD2VEC, w2v_path)
fasttext_model = StaticEmbedder(StaticEmbedder.Model.FASTTEXT, fasttext_path)
glove_model = StaticEmbedder(StaticEmbedder.Model.GLOVE, glove_path, binary=False)

In [ ]:
# Get some suggested collocations
treetagger_output = ["исследовать_V вопрос_N", "по_S мнение_N автор_N"]

for ngram, fixed_positions in zip(treetagger_output, [[1, 0], [1, 1, 0]]):
  query = ngram.lower().split(" ")
  
  w2v_results = w2v_model.suggest_collocations(query, fixed_positions, topn=5)
  fasttext_results = fasttext_model.suggest_collocations(query, fixed_positions, topn=5)
  glove_results = glove_model.suggest_collocations(query, fixed_positions, topn=25)
  
  print("Results for", query, ":")
  print("_____w2v_____")
  for res in w2v_results:
    print(res)
  print("\n_____fastText_____")
  for res in fasttext_results:
    print(res)
  print("\n_____GloVe_____")
  for res in glove_results:
    print(res)
  print("-"*75)

Results for ['исследовать_v', 'вопрос_n'] :
_____w2v_____
('исследовать проблема', 0, 0.6838977336883545)
('исследовать задача', 1, 0.48253941535949707)
('исследовать спор', 2, 0.459981232881546)
('исследовать вопрсы', 3, 0.44508951902389526)

_____fastText_____
('исследовать новопрос', 0, 0.8441401720046997)
('исследовать вопроа', 3, 0.800765872001648)

_____GloVe_____
('исследовать итервьюера', 23, 0.5414384603500366)
---------------------------------------------------------------------------
Results for ['по_s', 'мнение_n', 'автор_n'] :
_____w2v_____
('по мнение исследователь', 0, 0.6782662272453308)
('по мнение ученый', 2, 0.5683425664901733)
('по мнение историк', 3, 0.443101167678833)

_____fastText_____
('по мнение исследователь', 0, 0.6990058422088623)
('по мнение втор', 1, 0.6199074387550354)
('по мнение ученый', 2, 0.6077357530593872)
('по мнение пвтор', 3, 0.5964410305023193)

_____GloVe_____
('по мнение помпонио', 1, 0.7873562574386597)
('по мнение ряженки', 2, 0.74589282274

## Getting suggested collocations from dynamic word embeddings (RoBERTa, t5)

In [ ]:
from src.dynamic_embedder import DynamicEmbedder

bert_path = "/content/drive/MyDrive/models/lemma/RuBERT_treetagger_lemma"

# Load a dynamic word embedding model trained on a treetagger-lemmatized corpus
bert_model = DynamicEmbedder(DynamicEmbedder.Model.BERT, bert_path)

In [ ]:
# Get some suggested collocations
treetagger_output = ["исследовать_V <mask>_N", 
                     "по_S мнение_N <mask>_N", 
                     "рассматривать_V <mask>_N <mask>_N"]

for ngram in treetagger_output:
  query = ngram.lower()
  bert_results = bert_model.suggest_collocations(query, topn=5)
  
  print("Results for", query, ":")
  print("_____RoBERTa_____")
  for res in bert_results:
    print(res)
  print("-"*75)

Results for исследовать_v <mask>_n :
_____RoBERTa_____
('исследовать конституция', 0)
('исследовать законодательство', 1)
('исследовать автореферат', 2)
('исследовать диссертация', 3)
('исследовать закон', 4)
---------------------------------------------------------------------------
Results for по_s мнение_n <mask>_n :
_____RoBERTa_____
('по мнение NUMarab', 0)
('по мнение автор', 1)
('по мнение ученый', 2)
('по мнение эксперт', 3)
('по мнение судья', 4)
---------------------------------------------------------------------------
Results for рассматривать_v <mask>_n <mask>_n :
_____RoBERTa_____
('рассматривать международный NUMarab', 0)
('рассматривать международный монография', 1)
('рассматривать международный проблема', 2)
('рассматривать международный суд', 3)
('рассматривать международный целесообразный', 4)
('рассматривать содержание NUMarab', 1)
('рассматривать содержание монография', 2)
('рассматривать содержание проблема', 3)
('рассматривать содержание суд', 4)
('рассматривать 

## Getting suggested collocations & statistics

In [ ]:
# Run this and allow the IP on the test gcloud server 
!curl ipecho.net/plain

In [4]:
from src.get_collocation_replacements import getCollocationReplacements
from src.dynamic_embedder import DynamicEmbedder
from src.static_embedder import LossLogger, StaticEmbedder

bigrams = [l.lower() for l in ["исследовать_V вопрос_N", "школа_N экономика_N"]]
trigrams = [l.lower() for l in ["прийти_V к_S вывод_N", "по_S мнение_N автор_N"]]

w2v_path = "/content/drive/MyDrive/models/lemma/w2v/w2v_treetagger.model"
bert_path = "/content/drive/MyDrive/models/lemma/RuBERT_treetagger_lemma"

In [10]:
bigram_results = getCollocationReplacements(collocations=bigrams, 
                                            staticModel=True, 
                                            modelType=StaticEmbedder.Model.WORD2VEC,
                                            modelSrc=w2v_path, 
                                            binaryModel=True, # only relevant for GloVe
                                            cossim=True,
                                            topn=3,
                                            replace1=True, replace2=True, replace3=False,
                                            include_pmi=True, include_t_score=True, include_ngram_freq=True)

In [7]:
trigram_results = getCollocationReplacements(collocations=trigrams, 
                                             staticModel=False, 
                                             modelType=DynamicEmbedder.Model.BERT,
                                             modelSrc=bert_path,
                                             mask="<mask>",
                                             topn=3,
                                             replace1=True, replace2=True, replace3=True,
                                             include_pmi=True, include_t_score=True, include_ngram_freq=True)

In [11]:
def print_results(res):
  for k,v in res.items():
    print(f"Results for '{k}':")
    for val in v:
      print(" "*5, val)
    print("-"*75)

print("*"*5, "BIGRAMS WITH W2V", "*"*5)
print_results(bigram_results)
print("\n")
print("*"*5, "TRIGRAMS WITH BERT", "*"*5)
print_results(trigram_results)

***** BIGRAMS WITH W2V *****
Results for 'исследовать_v вопрос_n':
      {'suggested': 'анализировать вопрос', 'rank': 0, 'cosScore': 0.5687171816825867, 'numReplaced': 1, 'pmi': -1.9326116004215073, 't_score': -713.0812254264505, 'ngram_freq': 71}
      {'suggested': 'рассматривать вопрос', 'rank': 1, 'cosScore': 0.5575243234634399, 'numReplaced': 1, 'pmi': -0.8382014019729593, 't_score': -322.48555306741184, 'ngram_freq': 2998}
      {'suggested': 'изучить вопрос', 'rank': 2, 'cosScore': 0.553035318851471, 'numReplaced': 1, 'pmi': -1.5870879940255196, 't_score': -255.31760297908693, 'ngram_freq': 46}
      {'suggested': 'исследовать проблема', 'rank': 0, 'cosScore': 0.6838977336883545, 'numReplaced': 1, 'pmi': -0.722491713486314, 't_score': -132.48833138166378, 'ngram_freq': 959}
      {'suggested': 'анализировать вопрос', 'rank': 0, 'cosScore': 0.5687171816825867, 'numReplaced': 2, 'pmi': -1.9326116004215073, 't_score': -713.0812254264505, 'ngram_freq': 71}
      {'suggested': 'расс

## Putting it all together

In [ ]:
# Run this and allow the IP on the test gcloud server 
!curl ipecho.net/plain

In [2]:
from src.get_collocation_replacements import getCollocationReplacements
from src.dynamic_embedder import DynamicEmbedder
from src.tokenizer import Tokenizer

input_collocations = ["как мы полагаем", "по мнению авторов", "экономическое развитие страны"]
bert_path = "/content/drive/MyDrive/models/lemma/RuBERT_treetagger_lemma"

tokenizer = Tokenizer(Tokenizer.Method.TREETAGGER)
lemmas = [l.lower() for l in tokenizer.tokenize(input_collocations)]
res = getCollocationReplacements(collocations=lemmas, 
                                 staticModel=False, 
                                 modelType=DynamicEmbedder.Model.BERT,
                                 modelSrc=bert_path,
                                 mask="<mask>",
                                 topn=3,
                                 replace1=True, replace2=True, replace3=False,
                                 include_pmi=True, include_t_score=True, include_ngram_freq=True)
for k,v in res.items():
    print(f"Results for '{k}':")
    for val in v:
      print(" "*5, val)
    print("-"*75)

/usr/local/lib/python3.7/dist-packages/treetaggerwrapper.py:740: FutureWarning: Possible nested set at position 8
  re.IGNORECASE | re.VERBOSE)
/usr/local/lib/python3.7/dist-packages/treetaggerwrapper.py:2044: FutureWarning: Possible nested set at position 152
  re.VERBOSE | re.IGNORECASE)
/usr/local/lib/python3.7/dist-packages/treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
/usr/local/lib/python3.7/dist-packages/treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


Results for 'как_c мы_p полагать_v':
      {'suggested': 'как мы полагать', 'rank': 0, 'numReplaced': 1, 'pmi': -0.15771885657613424, 't_score': -4.7159769128401825, 'ngram_freq': 116}
      {'suggested': 'поэтому мы полагать', 'rank': 2, 'numReplaced': 1, 'pmi': -0.2828475279561056, 't_score': -4.001448538290557, 'ngram_freq': 19}
      {'suggested': 'как мы полагать', 'rank': 0, 'numReplaced': 1, 'pmi': -0.15771885657613424, 't_score': -4.7159769128401825, 'ngram_freq': 116}
      {'suggested': 'как он полагать', 'rank': 2, 'numReplaced': 1, 'pmi': -0.7479600608679128, 't_score': -21.066381231921344, 'ngram_freq': 21}
      {'suggested': 'как мы представляться', 'rank': 0, 'numReplaced': 1, 'pmi': 0.32084496077632957, 't_score': 11.502469224085171, 'ngram_freq': 485}
      {'suggested': 'как мы отмечать', 'rank': 1, 'numReplaced': 1, 'pmi': -0.7348097952799268, 't_score': -40.116484488604144, 'ngram_freq': 82}
      {'suggested': 'как мы видеть', 'rank': 2, 'numReplaced': 1, 'pmi': 0

## Save the results to file

In [3]:
from src.get_collocation_replacements import writeResults

writeResults(res, save_folder="/content/drive/MyDrive/ru_col_suggest/data/log")